In [1]:
import torch

In [17]:
byrd_train = torch.load("data/byrd/region_lower_byrd_train_tensor.pt", weights_only = False)
byrd_test = torch.load("data/byrd/region_lower_byrd_test_tensor.pt", weights_only = False)

In [18]:
train_input = byrd_train[0:3, :].T
test_input = byrd_test[0:3, :].T

In [16]:
import torch
import torch.nn as nn

class dfNN_aux(nn.Module):
    def __init__(self, input_coordinate_dim = 2, input_dim = 3, hidden_dim = 32):
        super().__init__()
        # Number of input coordinates (e.g., x, y)
        self.input_coordinate_dim = input_coordinate_dim
        # Number of input features (e.g., x, y, s) INCLUDING auxiliary inputs
        self.input_dim = input_dim
        self.output_dim = 1  # Scalar potential
        self.hidden_dim = hidden_dim

        # HACK: SiLu() worked much better than ReLU() for this model

        self.net = nn.Sequential(
            nn.Linear(self.input_dim, self.hidden_dim),
            nn.SiLU(),
            nn.Linear(self.hidden_dim, self.hidden_dim),
            nn.SiLU(),
            nn.Linear(self.hidden_dim, self.output_dim),
        )

    def forward(self, x):
        """
        Turn x1, x2 locations into vector fields
        x: [batch_size, input_dim]
        Returns: [batch_size, input_dim]  # Symplectic gradient
        """
        # Retrieve scalar potential
        H = self.net(x)

        # Compute full partials with respect to x coordinates, but only use coordinate dims
        partials = torch.autograd.grad(
                outputs = H.sum(), # we can sum here because every H row only depend on every x row
                inputs = x,
                create_graph = True
            )[0]
        
        # Only keep coordinate partials
        coord_partials = partials[:, :self.input_coordinate_dim]

        # Symplectic gradient
        # flip columns (last dim) for x2, x1 order. Multiply x2 by -1
        symp = coord_partials.flip(-1) * torch.tensor([1, -1], dtype = torch.float32, device = x.device)

        # return symp, H # NOTE: return H as well if we want to see what is going on
        return symp
        
model = dfNN_aux()
train_input.requires_grad = True  # Enable gradients for input
model(train_input)  # Should return [batch_size, 1] for scalar potential

tensor([[0.0285, 0.0519],
        [0.0285, 0.0534],
        [0.0285, 0.0533],
        [0.0285, 0.0531],
        [0.0284, 0.0530],
        [0.0283, 0.0530],
        [0.0282, 0.0530],
        [0.0281, 0.0529],
        [0.0281, 0.0528],
        [0.0280, 0.0527],
        [0.0279, 0.0526],
        [0.0278, 0.0526],
        [0.0277, 0.0526],
        [0.0275, 0.0527],
        [0.0274, 0.0526],
        [0.0273, 0.0526],
        [0.0272, 0.0525],
        [0.0271, 0.0525],
        [0.0270, 0.0524],
        [0.0270, 0.0524],
        [0.0269, 0.0523],
        [0.0268, 0.0522],
        [0.0261, 0.0518],
        [0.0260, 0.0518],
        [0.0259, 0.0518],
        [0.0258, 0.0518],
        [0.0257, 0.0517],
        [0.0289, 0.0510],
        [0.0290, 0.0509],
        [0.0291, 0.0509],
        [0.0292, 0.0509],
        [0.0293, 0.0510],
        [0.0294, 0.0510],
        [0.0294, 0.0510],
        [0.0295, 0.0511],
        [0.0296, 0.0511],
        [0.0297, 0.0511],
        [0.0298, 0.0511],
        [0.0

- Run across full domain
- Compare to PINNs
- Compare to unconstrained
- Boost of surface
- "Enforcing Local Mass Conservation in vector fields"
- underPINNed: Rethinking Physics-Informed Learning for Ice Sheet Flow 
Other idea: (flow control, Flux In, Flux Out)
New:
    - Directional guidance with partial observations from satellites
    - Incorporating surface predictors (auxiliary) from satellites
Compare such hard-constrained models with softconstrained PINNS and unconstrained NNs.


NN
PINN
dfNN

Vis: model
Vis: train & test: Predictions + div background